In [1]:
#Dn, n>=3
n=4 #lattice dim
m=n #space dim
L = [[1,1]+[0]*(n-2)] #a basis for the lattice, rows
for i in range(n-1):
    L.append([0]*i+[1,-1]+[0]*(n-2-i))

lat_det=abs(matrix(L).det())    
    
#covering radius
R=1 #if n=3
if n>3:
    R=sqrt(n)/2


#reduced form: non-increasing, non-negative
def reduce(x):
    y = [abs(t) for t in x]
    y.sort(key=lambda t: -t)
    return(y)

#first we generate all nodes with norm <=4R
short = []
K = ceil(4*R)
K2 = K^2
for x in mrange_iter([range(K+1) for i in range(n)]):
    descending = True
    for i in range(n-1):
        if x[i]<x[i+1]:
            descending = False
            break
    if descending and sum(x)%2==0 and sum(t^2 for t in x)<=K2 and x[0]>0:
        short.append(list(x))

print("short:",len(short))


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = []
for i in range(n-1):
    toadd = [0]*(n+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd)
ineqs.append([0]*n+[1])

#now Voronoi cell planes
for p in short:
    ineqs.append([sum((p[i])^2 for i in range(n))]+[-2*p[i] for i in range(n)])

V = Polyhedron(ieqs=ineqs)
    
### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))


### forbd = forb with possible permutations and signs
forbd = {}
forbdv = []
for x in forb:
    for k in mrange_iter([[-1,1] for i in range(n)]): #sign changes
        y = [x[j]*k[j] for j in range(n)]
        for t in Permutations(y):
            vt = vector(t)
            if str(vt) not in forbd:
                forbd[str(vt)]=0
                if sum(t)>=0:
                    forbdv.append(vt)
print("forbidden without symmetries:",len(forbd))
###

fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    A = A.transpose() #columns
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0



#exhaustive sublattice search
#each type of a matrix is the numbers on the diagonal
maxd = 49 #bound for diagonal entries
mindet = 31 #minimum determinant
maxdet = 49 #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]

allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

from time import time

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            inds = []
            l = i
            for j in range(n-1):
                inds.append(l%maxinds[j])
                l=floor(l/maxinds[j])
            mat = sublat_mat(tt,inds)
            cs = check_sublattice(mat)
            if cs==0:
                cs = prod(tt)
                if cs<bestv:
                    print("sublattice basis:")
                    print(mat)
                    bestv = cs
                    best = [mat]
                elif cs==bestv:
                    best.append(mat)
        print("best so far:",len(best))
    print("best:",len(best))
    print("min index:",bestv)
    tm = time()-tm
    print("total time mins:",tm/60)
    print("seconds per case:",tm/allc)
    return best




print(exh_search())




short: 12
forbidden: 9
forbidden without symmetries: 408
square L2 norm of forbidden: 12
small: 354
types: 646
all cases: 2705323
type: [1, 1, 1, 31] 31
cases of this type: 29791
best so far: 0
type: [1, 1, 1, 32] 32
cases of this type: 32768
best so far: 0
type: [1, 1, 1, 33] 33
cases of this type: 35937
best so far: 0
type: [1, 1, 1, 34] 34
cases of this type: 39304
best so far: 0
type: [1, 1, 1, 35] 35
cases of this type: 42875
best so far: 0
type: [1, 1, 1, 36] 36
cases of this type: 46656
best so far: 0
type: [1, 1, 1, 37] 37
cases of this type: 50653
best so far: 0
type: [1, 1, 1, 38] 38
cases of this type: 54872
best so far: 0
type: [1, 1, 1, 39] 39
cases of this type: 59319
best so far: 0
type: [1, 1, 1, 40] 40
cases of this type: 64000
best so far: 0
type: [1, 1, 1, 41] 41
cases of this type: 68921
best so far: 0
type: [1, 1, 1, 42] 42
cases of this type: 74088
best so far: 0
type: [1, 1, 1, 43] 43
cases of this type: 79507
best so far: 0
type: [1, 1, 1, 44] 44
cases of this t

best so far: 16
type: [1, 2, 22, 1] 44
cases of this type: 968
best so far: 16
type: [1, 2, 23, 1] 46
cases of this type: 1058
best so far: 16
type: [1, 2, 24, 1] 48
cases of this type: 1152
best so far: 16
type: [1, 3, 1, 11] 33
cases of this type: 3993
best so far: 16
type: [1, 3, 1, 12] 36
cases of this type: 5184
best so far: 16
type: [1, 3, 1, 13] 39
cases of this type: 6591
best so far: 16
type: [1, 3, 1, 14] 42
cases of this type: 8232
best so far: 16
type: [1, 3, 1, 15] 45
cases of this type: 10125
best so far: 16
type: [1, 3, 1, 16] 48
cases of this type: 12288
best so far: 16
type: [1, 3, 2, 6] 36
cases of this type: 2592
best so far: 16
type: [1, 3, 2, 7] 42
cases of this type: 4116
best so far: 16
type: [1, 3, 2, 8] 48
cases of this type: 6144
best so far: 16
type: [1, 3, 3, 4] 36
cases of this type: 1728
best so far: 16
type: [1, 3, 3, 5] 45
cases of this type: 3375
best so far: 16
type: [1, 3, 4, 3] 36
cases of this type: 1296
best so far: 16
type: [1, 3, 4, 4] 48
cases o

best so far: 16
type: [1, 39, 1, 1] 39
cases of this type: 39
best so far: 16
type: [1, 40, 1, 1] 40
cases of this type: 40
best so far: 16
type: [1, 41, 1, 1] 41
cases of this type: 41
best so far: 16
type: [1, 42, 1, 1] 42
cases of this type: 42
best so far: 16
type: [1, 43, 1, 1] 43
cases of this type: 43
best so far: 16
type: [1, 44, 1, 1] 44
cases of this type: 44
best so far: 16
type: [1, 45, 1, 1] 45
cases of this type: 45
best so far: 16
type: [1, 46, 1, 1] 46
cases of this type: 46
best so far: 16
type: [1, 47, 1, 1] 47
cases of this type: 47
best so far: 16
type: [1, 48, 1, 1] 48
cases of this type: 48
best so far: 16
type: [1, 49, 1, 1] 49
cases of this type: 49
best so far: 16
type: [2, 1, 1, 16] 32
cases of this type: 4096
best so far: 16
type: [2, 1, 1, 17] 34
cases of this type: 4913
best so far: 16
type: [2, 1, 1, 18] 36
cases of this type: 5832
best so far: 16
type: [2, 1, 1, 19] 38
cases of this type: 6859
best so far: 16
type: [2, 1, 1, 20] 40
cases of this type: 800

best so far: 16
type: [3, 1, 5, 3] 45
cases of this type: 675
best so far: 16
type: [3, 1, 6, 2] 36
cases of this type: 288
best so far: 16
type: [3, 1, 7, 2] 42
cases of this type: 392
best so far: 16
type: [3, 1, 8, 2] 48
cases of this type: 512
best so far: 16
type: [3, 1, 11, 1] 33
cases of this type: 121
best so far: 16
type: [3, 1, 12, 1] 36
cases of this type: 144
best so far: 16
type: [3, 1, 13, 1] 39
cases of this type: 169
best so far: 16
type: [3, 1, 14, 1] 42
cases of this type: 196
best so far: 16
type: [3, 1, 15, 1] 45
cases of this type: 225
best so far: 16
type: [3, 1, 16, 1] 48
cases of this type: 256
best so far: 16
type: [3, 2, 1, 6] 36
cases of this type: 432
best so far: 16
type: [3, 2, 1, 7] 42
cases of this type: 686
best so far: 16
type: [3, 2, 1, 8] 48
cases of this type: 1024
best so far: 16
type: [3, 2, 2, 3] 36
cases of this type: 216
best so far: 16
type: [3, 2, 2, 4] 48
cases of this type: 512
best so far: 16
type: [3, 2, 3, 2] 36
cases of this type: 144
b

best so far: 16
type: [8, 1, 1, 5] 40
cases of this type: 125
best so far: 16
type: [8, 1, 1, 6] 48
cases of this type: 216
best so far: 16
type: [8, 1, 2, 2] 32
cases of this type: 32
best so far: 16
type: [8, 1, 2, 3] 48
cases of this type: 108
best so far: 16
type: [8, 1, 3, 2] 48
cases of this type: 72
best so far: 16
type: [8, 1, 4, 1] 32
cases of this type: 16
best so far: 16
type: [8, 1, 5, 1] 40
cases of this type: 25
best so far: 16
type: [8, 1, 6, 1] 48
cases of this type: 36
best so far: 16
type: [8, 2, 1, 2] 32
cases of this type: 16
best so far: 16
type: [8, 2, 1, 3] 48
cases of this type: 54
best so far: 16
type: [8, 2, 2, 1] 32
cases of this type: 8
best so far: 16
type: [8, 2, 3, 1] 48
cases of this type: 18
best so far: 16
type: [8, 3, 1, 2] 48
cases of this type: 24
best so far: 16
type: [8, 3, 2, 1] 48
cases of this type: 12
best so far: 16
type: [8, 4, 1, 1] 32
cases of this type: 4
best so far: 16
type: [8, 5, 1, 1] 40
cases of this type: 5
best so far: 16
type: [8